## Create a basic data versions dictionnary , choosing models "by hand"
### This does not allow to finely select e.g. realizations per variable. See sibling notebook for a more complete selection of datasets

## Also (see bottom)  : create an extract of an existing version dictionnary, by restricting the models list

### IPCC/AR6/WGI
### Stéphane Sénési - sept 2019 / febr 2021

In [1]:
import sys
import json
lib="/home/ssenesi/CS2"
sys.path.append(lib) 
from AR6_utils.ancillary  import feed_dic
from AR6_utils.mips_et_al import read_versions_dictionnary

CliMAF version = 1.2.13


CliMAF install => /home/ssenesi/climaf_installs/climaf_running
python => /prodigfs/ipslfs/dods/jservon/miniconda/envs/analyse_env_2.7/bin/python
---
Required softwares to run CliMAF => you are using the following versions/installations:
ncl 6.6.2 => /prodigfs/ipslfs/dods/jservon/miniconda/envs/analyse_env_2.7/bin/ncl
cdo 1.9.6 => /opt/nco/1.9/bin/cdo
nco (ncks) "4.3.6" => /usr/bin/ncks
ncdump fichier => /prodigfs/ipslfs/dods/jservon/miniconda/envs/analyse_env_2.7/bin/ncdump
Check stamping requirements
nco (ncatted) found -> /usr/bin/ncatted
convert found -> /usr/bin/convert
pdftk found -> /usr/bin/pdftk
exiv2 found -> /home/lvignon/bin/exiv2
---


Cache directory set to : /data/ssenesi/climafcache (use $CLIMAF_CACHE if set) 
Cache directory for remote data set to : /data/ssenesi/climafcache/remote_data (use $CLIMAF_REMOTE_CACHE if set) 
No reachable lmdz_SE library
Available macros read from ~/.climaf.macros are : ['eu_zonal_mean', 'eu_cross_section']


In [2]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:100% !important; }</style>"))

## Tell preferences on some facets, per model (here, we assume this has not to depend on variable)

In [3]:
preferred_grids={ "GFDL-CM4":"gr1"}
preferred_variants={ }
preferred_versions={ }

## Tell which is period is covered by piControl, for each model to use (here, we assume this has not to depend on variable)
### One can use CliMAF dataset's method 'explore' for investigating which periods are available

In [4]:
piControl_period={
    "CESM2-WACCM":"0001-0499",
    "CNRM-ESM2-1":"1850-2349",
    "GFDL-ESM4":"0001-0500",
    "IPSL-CM6A-LR" : "1850-3049", 
    "GFDL-CM4":"0151-0650",
}

## A function for creating a dict of basic values for facets : grid, variant, version,period for a list of models

In [5]:
def basic_data_versions(variable,models) :
    variables=[variable]
    experiments=["historical","ssp126","ssp245","ssp585"]
    #
    data_versions=dict()
    for model in models :
        grid=preferred_grids.get(model,"g*")
        variant=preferred_variants.get(model,"r1i1p1f*")
        version=preferred_versions.get(model,"latest")
        for variable in variables :
            # For SSPs, do not include the period
            quadruplet=(grid,variant,version,"*")
            for exp in experiments :
                feed_dic(data_versions,quadruplet,exp,variable,model)
            # For piControl, do include the period
            quadruplet=(grid,variant,version,piControl_period[model])
            feed_dic(data_versions,quadruplet,"piControl",variable,model)
    return data_versions


In [6]:
def write_versions_dic(d,tag) :
    jsfile="Data_versions_selection_%s.json"%tag
    with open(jsfile,"w") as f :
        json.dump(d,f,separators=(',', ': '),indent=3,ensure_ascii=True)
    print "Data versions dictionnary written as "+jsfile

## Create and write a basic versions dictionnary

In [7]:
models=piControl_period.keys()
d=basic_data_versions("pr",models)
write_versions_dic(d,"5models")

Data versions dictionnary written as Data_versions_selection_5models.json


# Create a dict for reproducing SOD figure 8.18 for evspsbl except for EC-Earth models (which data was retracted)

In [17]:
# Use original per experiment dict and merge all models
models_to_plot_by_var={
    "ssp585": ["BCC-CSM2-MR", "CAMS-CSM1-0","CanESM5", "CNRM-ESM2-1",
               "IPSL-CM6A-LR",
               "MIROC-ES2L","MIROC6", "UKESM1-0-LL","MRI-ESM2-0",
               "CESM2","CESM2-WACCM", "GFDL-CM4","GFDL-ESM4"],
    "ssp245": ["BCC-CSM2-MR", "CAMS-CSM1-0","CanESM5", "CNRM-ESM2-1",
               "IPSL-CM6A-LR",
               "MIROC-ES2L","MIROC6", "UKESM1-0-LL","MRI-ESM2-0", "HadGEM3-GC31-LL",
               "CESM2","CESM2-WACCM", "GFDL-CM4","GFDL-ESM4"],
    "ssp126": ["BCC-CSM2-MR", "CAMS-CSM1-0","CanESM5", "CNRM-ESM2-1",
               "IPSL-CM6A-LR",
               "MIROC-ES2L","MIROC6","UKESM1-0-LL","MRI-ESM2-0",
               "CESM2","CESM2-WACCM", "GFDL-ESM4"],
}
# Merge model names across SSPs in all_models
ens=set()
for e in models_to_plot_by_var :
    for m in models_to_plot_by_var[e] :
        ens.add(m)
all_models=list(ens)
all_models.sort()

# Read dictionnary used for the february 2019 version of the figure
full_dic=read_versions_dictionnary("20200219","./")

# Feed a dictionnary with same content, but restricted to models in all_models
short_dic=dict()
for m in all_models :
    for exp in ["ssp126","ssp245","ssp585", "piControl","historical"] :
        try :
            feed_dic(short_dic,full_dic[exp]["evspsbl"][m],exp,"evspsbl",m)
        except :
            print "No entry for %s and %s"%(exp,m)
write_versions_dic(short_dic,"20200219_short_ssp585_evspsbl")

No entry for ssp126 and GFDL-CM4
No entry for ssp126 and HadGEM3-GC31-LL
No entry for ssp585 and HadGEM3-GC31-LL
Data versions dictionnary written as Data_versions_selection_20200219_short_ssp585_evspsbl.json
